# Game of Thrones Case Study

In this notebook we'll look at the case of Game of Thrones dataset and explore it with networkx-neo4j.

First let's import the required libraries:

In [1]:
import sys
sys.path.append("../")

In [2]:
# pip install git+https://github.com/neo4j-graph-analytics/networkx-neo4j.git#egg=networkx-neo4j

from neo4j import GraphDatabase
import nxneo4j

Next we'll create an instance of the Neo4j driver

In [5]:
# You'll need to change these credentials to point to your own Neo4j Server
driver = GraphDatabase.driver( "bolt://localhost", auth=("neo4j", "neo"))

## Importing the Dataset

Before we run any algorithms we'll first import a Game of Thrones dataset. 
This dataset was curated by [Dr Andrew Beveridge](https://twitter.com/mathbeveridge?lang=en).

In [19]:
with driver.session() as session:
    session.run("""\
    CREATE CONSTRAINT ON (c:Character)
    ASSERT c.name IS UNIQUE
    """)
    
    session.run("""\
    LOAD CSV WITH HEADERS FROM "https://raw.githubusercontent.com/mathbeveridge/asoiaf/master/data/asoiaf-book1-edges.csv" AS row
    MERGE (src:Character {name: row.Source})
    MERGE (tgt:Character {name: row.Target})
    // relationship for the book
    MERGE (src)-[r:INTERACTS1]->(tgt)
    ON CREATE SET r.weight = toInt(row.weight), r.book=1
    """)
    
    session.run("""\
    LOAD CSV WITH HEADERS FROM "https://raw.githubusercontent.com/mathbeveridge/asoiaf/master/data/asoiaf-book2-edges.csv" AS row
    MERGE (src:Character {name: row.Source})
    MERGE (tgt:Character {name: row.Target})
    // relationship for the book
    MERGE (src)-[r:INTERACTS2]->(tgt)
    ON CREATE SET r.weight = toInt(row.weight), r.book=2
    """)  
    
    session.run("""\
    LOAD CSV WITH HEADERS FROM "https://raw.githubusercontent.com/mathbeveridge/asoiaf/master/data/asoiaf-book3-edges.csv" AS row
    MERGE (src:Character {name: row.Source})
    MERGE (tgt:Character {name: row.Target})
    // relationship for the book
    MERGE (src)-[r:INTERACTS3]->(tgt)
    ON CREATE SET r.weight = toInt(row.weight), r.book=3
    """)       
    
    session.run("""\
    LOAD CSV WITH HEADERS FROM "https://raw.githubusercontent.com/mathbeveridge/asoiaf/master/data/asoiaf-book45-edges.csv" AS row
    MERGE (src:Character {name: row.Source})
    MERGE (tgt:Character {name: row.Target})
    // relationship for the book
    MERGE (src)-[r:INTERACTS45]->(tgt)
    ON CREATE SET r.weight = toInt(row.weight), r.book=45
    """)           

## Configuring our graph

Next we’re going to create a map explaining the node labels, relationship types, and properties used in the Graph of Thrones.

In [6]:
config = {
    "node_label": "Character",
    "relationship_type": '*',
    "identifier_property": "name"
}
G = nxneo4j.Graph(driver, config)

We set:

* `node_label` to `Character` so that we’ll only consider nodes with that label
* `relationship_type` to `*` so that we’ll consider all relationship types in the graph
* `identifier_property` is the node property that we’ll use to identify each node from the networkx-neo4j API

## Centrality

Let's take a look at the centrality algorithms we have available to us.

### PageRank

We’ll start with the famous PageRank algorithm. Let’s find out who the most influential characters in Game of Thrones are:

In [21]:
sorted_pagerank = sorted(nxneo4j.centrality.pagerank(G).items(), key=lambda x: x[1], reverse=True)
for character, score in sorted_pagerank[:10]:
    print(character, score)

Jon-Snow 17.59690941991715
Tyrion-Lannister 17.568134839111227
Jaime-Lannister 13.92549902428906
Cersei-Lannister 13.40237983062404
Daenerys-Targaryen 12.499216950197138
Stannis-Baratheon 12.150398000141639
Arya-Stark 11.692111553013346
Robb-Stark 11.27772564644931
Eddard-Stark 10.683881175699216
Catelyn-Stark 10.619218401320087


Hopefully there aren’t too many surprises there!

### Betweenness centrality

We can also run betweenness centrality over the dataset. This algorithm will tell us which nodes are the most 'pivotal' i.e. how many of the shortest paths between pairs of characters must pass through them

In [22]:
sorted_bw = sorted(nxneo4j.centrality.betweenness_centrality(G).items(), key=lambda x: x[1], reverse=True)
for character, score in sorted_bw[:10]:
    print(character, score)

Jon-Snow 65395.26787165436
Tyrion-Lannister 50202.17398521849
Daenerys-Targaryen 39636.777186621155
Stannis-Baratheon 35984.211828633124
Theon-Greyjoy 35436.85268519104
Jaime-Lannister 32122.97661542459
Robert-Baratheon 31391.065251945027
Arya-Stark 29342.158530621575
Cersei-Lannister 28274.915426635573
Eddard-Stark 26470.25024909826


### Closeness centrality

Closeness centrality tells us on average how many hops away each character is from every other character.

In [23]:
sorted_cc = sorted(nxneo4j.centrality.closeness_centrality(G, wf_improved=False).items(), key=lambda x: x[1], reverse=True)
for character, score in sorted_cc[:10]:
    print(character, score)

Tyrion-Lannister 0.4763331336129419
Robert-Baratheon 0.4592720970537262
Eddard-Stark 0.455848623853211
Cersei-Lannister 0.45454545454545453
Jaime-Lannister 0.4519613416714042
Jon-Snow 0.44537815126050423
Stannis-Baratheon 0.4446308724832215
Robb-Stark 0.4441340782122905
Joffrey-Baratheon 0.4339519650655022
Catelyn-Stark 0.4334787350054526


Again we see the usual suspects ranking highly on this metric of centrality.

## Pathfinding 

Our next category of algorithms are used for path finding.

### Shortest Path

What if we want to find the shortest path between two characters?

In [25]:
nxneo4j.path_finding.shortest_path(G, "Tyrion-Lannister", "Hodor")

['Tyrion-Lannister', 'Luwin', 'Hodor']

Notice that we refer to nodes by their name property — this is where the `identifier_property` that we defined in our config map is used.

## Label Propagation

We can also partition the characters into communities using the label propagation algorithm:

In [26]:
communities = nxneo4j.community.label_propagation_communities(G)
sorted_communities = sorted(communities, key=lambda x: len(x), reverse=True)
for community in sorted_communities[:10]:
    print(list(community)[:10])

['Lyonel-Corbray', 'Martyn-Rivers', 'Franklyn-Fowler', 'Trystane-Martell', 'Captain-Myraham', 'Tytos-Blackwood', 'Simon-Toyne', 'Poxy-Tym', 'Robert-Strong', 'Tom-of-Sevenstreams']
['Quill', 'Chett', 'Watt', 'Haggon', 'Alliser-Thorne', 'Mallador-Locke', 'Weeper', 'Big-Boil', 'Aemon-Targaryen-(Maester-Aemon)', 'Dryn']
['Belaquo', 'Meris', 'Kedry', 'Gerris-Drinkwater', 'Aggo', 'Skahaz-mo-Kandaq', 'Mirri-Maz-Duur', 'Miklaz', 'Galazza-Galare', 'Symon-Stripeback']


Characters are in the same community as those other characters with whom they frequently interact. The idea is that characters have closer ties to those in their community than to those outside.

### Clustering

We can calculate the clustering coefficient for each character.
A clustering coefficient of '1' means that all characters that interact with that character also interact with each other:

In [7]:
biggest_coefficient = sorted(nxneo4j.community.clustering(G).items(), key=lambda x: x[1], reverse=True)
for character in biggest_coefficient[:10]:
    print(list(character)[:10])

['Steffon-Baratheon', 4.0]
['Oswell-Kettleblack', 4.0]
['Wylis-Manderly', 4.0]
['Beth-Cassel', 3.0]
['Big-Boil', 3.0]
['Dirk', 3.0]
['Jon-Umber-(Smalljon)', 3.0]
['Orell', 3.0]
['Oznak-zo-Pahl', 3.0]
['Mag-Mar-Tun-Doh-Weg', 3.0]


In [8]:
smallest_coefficient = sorted(nxneo4j.community.clustering(G).items(), key=lambda x: x[1])
for character in smallest_coefficient[:10]:
    print(list(character)[:10])

['William-Foxglove', 0.0]
['Clarence-Crabb', 0.0]
['Illifer', 0.0]
['Jeyne-Heddle', 0.0]
['Willow-Heddle', 0.0]
['Torghen-Flint', 0.0]
['Lucifer-Long', 0.0]
['Hugh-Hungerford', 0.0]
['Raymun-Redbeard', 0.0]
['Eustace-Brune', 0.0]
